<a href="https://colab.research.google.com/github/jfzhang726/Finetune-llama3/blob/main/finetune_llama3_kaggle_essay_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.getcwd()

# mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import torch
torch.__version__

'2.2.1+cu121'

In [2]:
%%capture

import torch
major_version, minor_version = torch.cuda.get_device_capability()

# Colab has torch 2.2.1 which will break the package, so install package seperatetly
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
  # new gpu Ampere, Hooper GPU (RTX 30xx, RTX 40xx,, A100, H100, L40)
  !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
  # old GPU V100, Tesla, T4, RTX 20xx
  !pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes
pass

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [4]:
# load dataset
import os
from datasets import load_dataset
EOS_TOKEN = tokenizer.eos_token

alpaca_prompt_essay_score = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.


### Instruction:
You are an English teacher. Given the following essay, you give a score.

### Input:
{input}

### Response:
{output}
"""

def formatting_prompts_func_train(examples):

  essay = examples["full_text"]
  score = examples["score"]
  texts = []
  for essay, score in zip(essay, score):
    # must add EOS_TOKEN, otherwise the generation never stops
    text = alpaca_prompt_essay_score.format(input=essay, output=f"Score: {score}") + EOS_TOKEN
    texts.append(text),
  return {'text': texts}


def formatting_prompts_func_test(examples):

  essay = examples["full_text"]
  score = examples["score"]
  texts = []
  for essay, score in zip(essay, score):
    # must add EOS_TOKEN, otherwise the generation never stops
    text = alpaca_prompt_essay_score.format(input=essay, output=f"Score:")
    texts.append(text),
  return {'text': texts}

data_path = "/content/drive/MyDrive/Colab Notebooks/kaggle_automated_essay_scoring_2/"
data_file = os.path.join(data_path, "train.csv")

dataset = load_dataset("csv", data_files=data_file, split='train')

train_test_split = dataset.train_test_split(test_size=0.2, seed=42)  # 80% train, 20% test

# Access the train and test sets
dataset_train = train_test_split['train']
dataset_test = train_test_split['test']
dataset_train = dataset_train.map(formatting_prompts_func_train, batched=True, batch_size=len(dataset_train))
dataset_test = dataset_test.map(formatting_prompts_func_test, batched=True, batch_size=len(dataset_train))

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/13845 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [5]:
# test before finetune


FastLanguageModel.for_inference(model)
inputs = tokenizer([
    dataset_test['text'][0]
], return_tensors='pt').to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.


### Instruction:
You are an English teacher. Given the following essay, you give a score.

### Input:
Based on the many differences of positive and negative comments towards the use of driverless cars, I wish to pick neither side in the disscusion.

I believe there are many reasons that driverless cars should take over the roads, but a handfull of reasons that driverless cars on our roads today would lead to dissaster.

To start off, there was a point made in paragraph one, where the author tells us that new smart cars or driverless cars would use half of the gas as todays taxis.

With this, there could be many things that move in a positive direction.

For example, if cars are running on half of the ammount of fuel that todays taxis use, then there is no doubt about the air polution in our cities and other coun

In [6]:
inputs = tokenizer([
    dataset_test['text'][1]
], return_tensors='pt').to("cuda")

_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.


### Instruction:
You are an English teacher. Given the following essay, you give a score.

### Input:
Transportation helps us get to where we need to go. One way of transportaion that most people use are cars. Cars have all changed thorughout decades. New equipment is put in a car to make them more fancy or more safe.

Technology is all around us now. How would it feel if a car controlled us? There are so many way to argue this topic about cars that control themselves. What is really the purpose of this car?

All of the cars made before the Driverless Cars were driven by a person. Although the driverless cars are not that popular yet, people are intrested in them. The more technology we use, it makes society seem more lazy. "- but how much smarter did the cars need to be?"(Driverless Cars Are Coming). This feels

In [7]:
# hyperparameters for training
from trl import SFTTrainer
from transformers import TrainingArguments

model = FastLanguageModel.get_peft_model(
    model,
    r=16, # 8, 16, 32, 64, 128
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing='unsloth',
    random_state=42,
    use_rslora=False,
  loftq_config=None

)



Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=409

In [8]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    dataset_text_field='text',
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False, # can speed up training of short sequences for 5 times
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=3,
        warmup_steps=5,
        num_train_epochs=1, # it will take 2+ hours for gsm8k
        #max_steps=60, # finetune steps, which will override num_train_epochs
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim='adamw_8bit',
        weight_decay=0.01,
        lr_scheduler_type='linear',
        seed=42,
        output_dir='outputs'
    )
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/13845 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 13,845 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 3
\        /    Total batch size = 6 | Total steps = 2,307
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.774700
2,2.609100
3,2.623000
4,2.934900
5,2.618500
6,2.751700
7,2.435100
8,2.556100
9,2.339300
10,2.393700


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

In [11]:
# test after finetuning
FastLanguageModel.for_inference(model)
print('label:', dataset_test['score'][0])
inputs = tokenizer([
    dataset_test['text'][0]
], return_tensors='pt').to("cuda")

_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


label: 3
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.


### Instruction:
You are an English teacher. Given the following essay, you give a score.

### Input:
Based on the many differences of positive and negative comments towards the use of driverless cars, I wish to pick neither side in the disscusion.

I believe there are many reasons that driverless cars should take over the roads, but a handfull of reasons that driverless cars on our roads today would lead to dissaster.

To start off, there was a point made in paragraph one, where the author tells us that new smart cars or driverless cars would use half of the gas as todays taxis.

With this, there could be many things that move in a positive direction.

For example, if cars are running on half of the ammount of fuel that todays taxis use, then there is no doubt about the air polution in our cities and o

In [12]:
print('label:', dataset_test['score'][1])
inputs = tokenizer([
    dataset_test['text'][1]
], return_tensors='pt').to("cuda")

_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


label: 3
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.


### Instruction:
You are an English teacher. Given the following essay, you give a score.

### Input:
Transportation helps us get to where we need to go. One way of transportaion that most people use are cars. Cars have all changed thorughout decades. New equipment is put in a car to make them more fancy or more safe.

Technology is all around us now. How would it feel if a car controlled us? There are so many way to argue this topic about cars that control themselves. What is really the purpose of this car?

All of the cars made before the Driverless Cars were driven by a person. Although the driverless cars are not that popular yet, people are intrested in them. The more technology we use, it makes society seem more lazy. "- but how much smarter did the cars need to be?"(Driverless Cars Are Coming). T

In [13]:
print('label:', dataset_test['score'][2])
inputs = tokenizer([
    dataset_test['text'][2]
], return_tensors='pt').to("cuda")

_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


label: 3
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.


### Instruction:
You are an English teacher. Given the following essay, you give a score.

### Input:
Cars are a main aspect in our daily lives, but how would it be if we limited car usage throughout the world? Places would be more environmentally friendly and the streets would be less congested. So why not give it a shot, what could possibly go wrong?

If we limited car usage, it would be easier to get to our destination. Places would be more compact with less space for parking so that stores would be built close enough for us to walk there rather than driving to the nearest mall. Less malls would mean less parking lots, meaning sotres would be built along main streets rather than a highway.

Not only does less cars make it easier to get to the places we need to be, but makes people happier. In the art

In [14]:
print('label:', dataset_test['score'][3])
inputs = tokenizer([
    dataset_test['text'][3]
], return_tensors='pt').to("cuda")

_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


label: 2
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.


### Instruction:
You are an English teacher. Given the following essay, you give a score.

### Input:
A Cowboy Who Rode he Waves Essay

Intoduction

Luke had turned 18 before he arrived in Greece, he said" I could be drafted for military services" ( paragraph 4). It took Luke to get across the Atlantic Ocean eastern coast of the United States 2 weeks, and 1 month to get to China. Luke had made a least 9 trips, more than any Seagoing cowboy. By the time it was 1947 Luke had helped people he said " Besides helping people, I had the side benefit of seeing Europe and China" ( paragraph 5). Luke had visited Venice,Italy he rode a gondola because the streets were water. He also " took a tour in a excavated castle in Crete, and the Panama Canalon his way to China" (paragraph 5).

Reasons A few reasons why you 

In [15]:
print('label:', dataset_test['score'][4])
inputs = tokenizer([
    dataset_test['text'][4]
], return_tensors='pt').to("cuda")

_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


label: 3
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.


### Instruction:
You are an English teacher. Given the following essay, you give a score.

### Input:
We should keep the elcetoral college because it was made by out founding fathers and should be kept alive. Not only that but it has been around for so long that it would be difficult getting use to the change. By keeping the electoral college we keep a non-democratic way of voting for out presidents. Betrayal dosent often happen because we put trust into the people we sent to the electoral college. By keeping the electoral college we also keep the chance of any runner winning, unlike if normal people got to have their votes count someone would always ain by a landslide. By also keeping the electoral college we make it so their is no chance of a tie. Like if a person from the electoral college were to be

In [16]:
print('label:', dataset_test['score'][5])
inputs = tokenizer([
    dataset_test['text'][5]
], return_tensors='pt').to("cuda")

_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


label: 3
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.


### Instruction:
You are an English teacher. Given the following essay, you give a score.

### Input:
I'm agaisn't the Driverless Cars Are Coming because I feel that it is not safe at all.You don't know what the driveless car is really accapable of, other then what they tell you. Driveless cars won't be able to tell when an accident is occuring? How could our technology is that devloped or advanced? It's not. It would take over a billion of years to be very advanced enough to be on the road.

I don't even think sensors would even help for this car to be safe. What if they sensors are not picking up something important than your suck? Maybe even in the middle of a train track because they senes to stop right away. Not all technology is good technology. I don't believe that sensors are advancing to where 

In [17]:
# save LoRA model

model.save_pretrained("lora_model_essay_scoring") # save to local drive
# model.push_to_hub("jfzhang/lora_model", token='xxxx') # save to huggingface

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
# merge lora model and base model, quantilize to 4-bit, save gguf
model.save_pretrained_gguf("model_essay_scoring", tokenizer, quantization_method='q4_k_m')

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.8 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:43<00:00,  3.22s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model_essay_scoring/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model_essay_scoring/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model_essay_scoring/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model_essay_scoring/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to q4_k_m will take 20 minutes.
 "-____-"     In total, you will have to wait around 26 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model_essay_scoring into f16 GGUF format.
The output location will be ./model_essay_scoring-unsloth.F16.gguf
This will take 3 minutes...
Unsloth: Conversion completed! Output location: ./model_essay_scoring-unsloth.F16.gguf
Unsloth: [2] Converting GGUF 16bit into q4_k_m. This will take 20 minutes...
main: build = 2786 (03fb8a00)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing './model_essay_scoring-unsloth.F16.gguf' to './model_essay_scoring-unsloth.Q4_K_M.gguf' as Q4_K_M using 4 threads
llama_model_loader: loaded meta

'/content'

Mounted at /content/drive/


In [19]:
# copy model file to google drive
import shutil
import os
source_file = '/content/model_essay_scoring-unsloth.Q4_K_M.gguf'
destination_dir = '/content/drive/MyDrive/Colab Notebooks/unsloth/Llama3'
destination_file = os.path.join(destination_dir, 'model_essay_scoring-unsloth.Q4_K_M.gguf')
shutil.copy(source_file, destination_file)

'/content/drive/MyDrive/Colab Notebooks/unsloth/Llama3/model_essay_scoring-unsloth.Q4_K_M.gguf'

In [30]:
source_file = '/content/model_essay_scoring-unsloth.F16.gguf'
destination_dir = '/content/drive/MyDrive/Colab Notebooks/unsloth/Llama3'
destination_file = os.path.join(destination_dir, 'model_essay_scoring-unsloth.F16.gguf')
shutil.copy(source_file, destination_file)

OSError: [Errno 28] No space left on device: '/content/drive/MyDrive/Colab Notebooks/unsloth/Llama3/model_essay_scoring-unsloth.F16.gguf'

In [ ]:
# evaluation with test dataset
import re
# find number followed by ### Response:\n\nScore:\n\n


def extrat_score(responses):
    if type(responses) != list:
        responses = [responses]
    scores = []
    pattern = re.compile(r'### Response:\s+Score:\s*(\d)')
    for r in responses:
        match = re.search(pattern, r)
        score = match.group(1) if match else None
        scores.append(score)

    return scores

FastLanguageModel.for_inference(model)

results = []
count = 0
for idx, sample in enumerate(dataset_test):
    # print(idx, sample['score'], sample['text'])
    text = sample['text']
    score = sample['score']
    inputs = tokenizer([
        text
    ], return_tensors='pt').to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
    outputs = tokenizer.batch_decode(outputs)
    predictions = extrat_score(outputs)
    result = {k: v for k, v in sample.items()}
    result['marking'] = outputs[0]
    result['prediction'] = predictions[0]
    results.append(result)
    count += 1
    if count % 100 ==0:
      print(count)
    if count > 100:
      break
destination_dir = '/content/drive/MyDrive/Colab Notebooks/unsloth/Llama3'

import json
with open(os.path.join(destination_dir, 'auto-scoring', 'results.json'), 'w') as f:
    json.dump(results, f)

correct_count = 0
for r in results:
    if r['prediction'] and int(r['prediction']==r['score']):
        correct_count += 1
print(f"rate: {correct_count/len(results)}")

# date time suffix
from datetime import datetime
suffix = datetime.now().strftime("%Y%m%d-%H%M%S")
# write rate into file
with open(os.path.join(destination_dir, 'auto-scoring', f'experiment_{suffix}.txt')) as f:
    f.write(f"rate: {correct_count/len(results)}")
    f.write('\n')


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [28]:
destination_dir = '/content/drive/MyDrive/Colab Notebooks/unsloth/Llama3'

import json
with open(os.path.join(destination_dir, 'auto-scoring', 'results.json'), 'w') as f:
    json.dump(results, f)

correct_count = 0
for r in results:
    if r['prediction'] and int(r['prediction'])==r['score']:
        correct_count += 1
print(f"rate: {correct_count/len(results)}")

# date time suffix
from datetime import datetime
suffix = datetime.now().strftime("%Y%m%d-%H%M%S")
# write rate into file
with open(os.path.join(destination_dir, 'auto-scoring', f'experiment_{suffix}.txt'), 'w') as f:
    f.write(f"rate: {correct_count/len(results)}")
    f.write('\n')

rate: 0.19801980198019803


In [27]:
[(r['score'], r['prediction']) for r in results]

[(3, '2'),
 (3, '2'),
 (3, '2'),
 (2, '2'),
 (3, '2'),
 (3, '2'),
 (2, '2'),
 (1, '2'),
 (1, '2'),
 (3, '2'),
 (2, '2'),
 (4, '2'),
 (1, '2'),
 (4, '2'),
 (5, '3'),
 (1, '2'),
 (4, '2'),
 (4, '2'),
 (2, '2'),
 (4, '2'),
 (5, '3'),
 (3, '2'),
 (2, '2'),
 (3, '2'),
 (3, '2'),
 (3, '2'),
 (4, '2'),
 (2, '2'),
 (5, '2'),
 (4, '3'),
 (3, '2'),
 (2, '2'),
 (3, '2'),
 (4, '2'),
 (3, '2'),
 (3, '2'),
 (5, '2'),
 (3, '2'),
 (4, '2'),
 (5, '3'),
 (3, '2'),
 (2, '2'),
 (4, '2'),
 (4, '2'),
 (3, '2'),
 (4, '2'),
 (4, '2'),
 (4, '3'),
 (3, '2'),
 (4, '2'),
 (2, '2'),
 (3, '2'),
 (3, '2'),
 (4, '2'),
 (3, '2'),
 (1, '2'),
 (3, '2'),
 (3, '2'),
 (2, '2'),
 (4, '3'),
 (3, '2'),
 (4, '2'),
 (4, '2'),
 (2, '2'),
 (4, '2'),
 (3, '2'),
 (3, '2'),
 (4, '2'),
 (3, '2'),
 (2, '2'),
 (3, '2'),
 (1, '2'),
 (3, '2'),
 (4, '2'),
 (3, '2'),
 (4, '2'),
 (2, '2'),
 (2, '2'),
 (3, '2'),
 (2, '2'),
 (2, '2'),
 (1, '2'),
 (1, '2'),
 (3, '2'),
 (2, '2'),
 (2, '2'),
 (5, '2'),
 (2, '2'),
 (5, '2'),
 (3, '2'),
 (6, '2'),